In [3]:
import sys
import json
sys.path.append('..')
from src.common import *
from src.analysis.model_performances import *
from copy import deepcopy
from helpers import *
import pandas as pd


TO_PRETTY |= {(WITHOUT_RANDOM_SUB, WITHOUT_RAMIFICATIONS): 'Baseline', 
              (WITHOUT_RANDOM_SUB, WITH_RAMIFICATIONS): 'Baseline + R.',
             (WITH_RANDOM_SUB, WITHOUT_RAMIFICATIONS): 'Obfus. Baseline',
             (WITH_RANDOM_SUB, WITH_RAMIFICATIONS): 'Obfus. Baseline + R.',
}

In [4]:
answer_type = TRUE_FALSE_ANSWER_TYPE #FREE_ANSWER#
score_key = ACCURACY_SCORE_KEY #F1_SCORE_KEY#'accuracy'
answer_type_ext = tf_answer_type(score_key = score_key)

ids_file_name = 'dataset_ids.test.pruned'  # None
save_main_dir = f'{STATISTICS_PATH}.{ids_file_name}'
stats_all = collect_stats_all(tf_answer_type(score_key = score_key), save_main_dir=save_main_dir)
print(len(stats_all))
plan_length= 19

save_dir = os.path.join(save_main_dir, 'tables', 'by_categories')
os.makedirs(save_dir, exist_ok=True)


100%|██████████| 48384/48384 [00:02<00:00, 23125.77it/s]

5154


In [5]:
def to_df_by_category(results_all, answer_type,  
                      model_name,
                      prompt_type='few_shot_5',
                      domain = ALL_DOMAINS_KEY,
                      plan_length=19):

    index = []
    data = []    
    for question_category in QUESTION_CATEGORIES+[ALL_QUESTION_CATEGORIES_KEY]:
        index.append(TO_PRETTY[question_category])
        data_columns = {}
        for subs in [WITHOUT_RANDOM_SUB, WITH_RANDOM_SUB]:
            for ramifications in [WITHOUT_RAMIFICATIONS, WITH_RAMIFICATIONS]:
                res_obj = filter_single_selector(results_all, plan_length, question_category, ramifications, model_name, prompt_type, domain, answer_type, subs)
                if res_obj:
                    mean = res_obj['result']
                    sem = None
                    if res_obj['result_other']:
                        sem = res_obj['result_other']['sem']
                    not_corrupted = res_obj['stats']['num_not_corrupted']
                    final_res = (mean, sem, not_corrupted)
                else:
                    final_res = (None, None, None)
                final_res = tuple([round(v*100, 2) if v else v for v in final_res ])
                final_res = '${'+str(final_res[0])+'}_{'+str(final_res[1])+'}$'
                data_columns[TO_PRETTY[(subs, ramifications)]] = final_res
        data.append(data_columns)
    return pd.DataFrame(data, index = index)

# Subs and Ramfications

In [6]:
plan_length = 19
model_name = 'gemini'
prompt_type = 'few_shot_1'
df = to_df_by_category(stats_all, answer_type,model_name=model_name, prompt_type=prompt_type, plan_length=plan_length)
df
        
# caption_nl = f'performance of {model_name}, {prompt_type}, pl-{plan_length}'.replace('_', ' ')
# save_key = f'{model_name}.{prompt_type}.{plan_length}'
# 
# latex_table_all = to_latex_table(df, caption_nl, label=save_key)
# with open(os.path.join(save_dir, f'{save_key}.tex'), 'w') as f:
#     f.write(latex_table_all)

,Baseline,Baseline + R.,Obfus. Baseline,Obfus. Baseline + R.
Object Trk.,${67.42}_{2.04}$,${68.38}_{2.03}$,${65.91}_{2.06}$,${68.94}_{2.01}$
Fluent Trk.,${58.75}_{2.17}$,${59.57}_{2.17}$,${53.11}_{2.2}$,${56.03}_{2.19}$
State Trk.,${59.09}_{7.41}$,${59.09}_{7.41}$,${47.73}_{7.53}$,${47.73}_{7.53}$
Action Exec.,${52.5}_{5.58}$,${53.75}_{5.57}$,${52.5}_{5.58}$,${51.25}_{5.59}$
Effects,${54.89}_{2.79}$,${54.57}_{2.8}$,${50.16}_{2.81}$,${50.47}_{2.81}$
Num. Reas.,${48.75}_{5.59}$,${52.5}_{5.58}$,${51.25}_{5.59}$,${50.0}_{5.59}$
Hallucination,${78.75}_{4.57}$,${88.75}_{3.53}$,${71.25}_{5.06}$,${67.5}_{5.24}$
Composite,${None}_{None}$,${None}_{None}$,${None}_{None}$,${None}_{None}$
AVG,${60.99}_{1.2}$,${62.21}_{1.2}$,${57.27}_{1.22}$,${58.92}_{1.21}$


# Fluent Types 